In [1]:
!pip install --quiet -U pip
!pip install --quiet -U openai-whisper
!pip install feedparser
!pip install --quiet -U sentence-transformers
!apt-get update && apt install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=0a5a9714d2f66fee59e1b3c09368bd7aca20db1037dabc6d8ac4d135a0734320
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]       
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [2217 B]
Get:4 https://packages.cloud.google.com/apt cloud-sdk/main amd64 Packages [430 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease                        
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu

In [4]:
!pip uninstall whisper
!pip install --force-reinstall openai-whisper==20230124

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 20.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 8.2 MB/s eta 0:00:00
  Using cached more_itertools-9.1.0-py3-none-any.whl (54 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.3 MB/s eta 0:00:0000:0100:01
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import re
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.signal import argrelextrema
import math

import whisper

import feedparser
import urllib.request

### transcription model ###
model_size = "large-v2"

print('Loading transcription model...')
# Run on GPU with FP16
model = whisper.load_model(model_size) #, device="cuda")
# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")
print('Done.')

### paragraph model ###
print('Loading word embedding model...')
sentencemodel = SentenceTransformer('all-mpnet-base-v2')
print('Done.')

def rev_sigmoid(x:float)->float:
    return (1 / (1 + math.exp(0.5*x)))
    
def activate_similarities(similarities:np.array, p_size=10)->np.array:
        """ Function returns list of weighted sums of activated sentence similarities
        Args:
            similarities (numpy array): it should square matrix where each sentence corresponds to another with cosine similarity
            p_size (int): number of sentences are used to calculate weighted sum 
        Returns:
            list: list of weighted sums
        """
        # To create weights for sigmoid function we first have to create space. P_size will determine number of sentences used and the size of weights vector.
        x = np.linspace(-10,10,p_size)
        # Then we need to apply activation function to the created space
        y = np.vectorize(rev_sigmoid) 
        # Because we only apply activation to p_size number of sentences we have to add zeros to neglect the effect of every additional sentence and to match the length ofvector we will multiply
        activation_weights = np.pad(y(x),(0,similarities.shape[0]-p_size))
        ### 1. Take each diagonal to the right of the main diagonal
        diagonals = [similarities.diagonal(each) for each in range(0,similarities.shape[0])]
        ### 2. Pad each diagonal by zeros at the end. Because each diagonal is different length we should pad it with zeros at the end
        diagonals = [np.pad(each, (0,similarities.shape[0]-len(each))) for each in diagonals]
        ### 3. Stack those diagonals into new matrix
        diagonals = np.stack(diagonals)
        ### 4. Apply activation weights to each row. Multiply similarities with our activation.
        diagonals = diagonals * activation_weights.reshape(-1,1)
        ### 5. Calculate the weighted sum of activated similarities
        activated_similarities = np.sum(diagonals, axis=0)
        return activated_similarities

def paragraphise(bigstring):
    sentences = bigstring.split('. ')
    
    embeddings = sentencemodel.encode(sentences, show_progress_bar=False)
    similarities = cosine_similarity(embeddings)
    activated_similarities = activate_similarities(similarities, p_size=5)
    minimas = argrelextrema(activated_similarities, np.less, order=2) #order parameter controls how frequent should be splits. I would not reccomend changing this parameter.

    split_points = [each for each in minimas[0]]
    text = ''
    for num,each in enumerate(sentences):
        if num in split_points:
            text+=f'\n\n {each}. '
        else:
            text+=f'{each}. '
    return text

### other functions ### 
def zfill_alternative(x,l=2): return x if len(x) >= l else '0'*(l-len(x))+x

Loading transcription model...
Done.
Loading word embedding model...
Done.


In [7]:
import feedparser
import urllib.request
NewsFeed = feedparser.parse("https://tehnikarechi.studio/api/rss/podcasts/knizhnyy-bazar")
entry = NewsFeed.entries[0]

print(entry.keys())

dict_keys(['title', 'title_detail', 'id', 'guidislink', 'links', 'link', 'summary', 'summary_detail', 'content', 'published', 'published_parsed', 'authors', 'author', 'author_detail', 'image', 'itunes_duration', 'subtitle', 'subtitle_detail', 'itunes_episodetype', 'tags', 'itunes_explicit'])


In [7]:
!mkdir knizhnyy-bazar

In [8]:
NewsFeed = feedparser.parse("https://tehnikarechi.studio/api/rss/podcasts/knizhnyy-bazar")

dl_dir = 'knizhnyy-bazar'

failsforsomereason = []

arr = os.listdir('/kaggle/working/'+dl_dir)

for entry in tqdm(NewsFeed.entries):
    notdone = True
    while notdone:
        try:
            vid_title = re.sub(r'[^A-Za-zЁёА-я0-9— ]+', '', entry['title'].replace(u'\xa0', u' '))[:130]

            if (vid_title in [x[:-4] for x in arr] or 
                vid_title in failsforsomereason):
                notdone = False
                continue

            opener = urllib.request.build_opener()
            opener.addheaders = [('User-agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(entry['links'][1]['href'], "audio.mp3")
            print(vid_title)

            #print(yt.streams.filter(only_audio=True).order_by('abr'))

            transcription = model.transcribe("audio.mp3",  language="ru")
            
            prettytext = paragraphise(transcription['text'])

            with open(dl_dir+'/'+vid_title+'.txt','w+',encoding='utf-8') as myfile:
                myfile.write('---')
                myfile.write('\ntitle: '+vid_title)
                myfile.write('\nauthor: '+entry['author'])
                myfile.write('\npublished: '+entry['published'])
                myfile.write('\ntags: '+str([ a['term'] for a in entry['tags']]))
                myfile.write('\n---\n\n')
                myfile.write(prettytext)
            
            print('Done')
            notdone = False
        except Exception as e:
            continue

  0%|          | 0/106 [00:00<?, ?it/s]

Глава в которой Галя и Настя отвечают на ваши вопросы и спойлерят следующий сезон


/opt/conda/lib/python3.7/site-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
 54%|█████▍    | 57/106 [2:40:50<2:18:15, 169.30s/it]


KeyboardInterrupt: 

In [12]:
!tar -zcvf knizhnyy-bazar.tar.gz knizhnyy-bazar/*.txt

knigtok/5 февраля начнется новый сезон Книжного базара — о книгах на которых мы все выросли Что перечитать Мы составили список.txt
knigtok/Глава в которой Аэлиту съедает Чужой Пелевин обнуляет космос но всех спасает Илон Маск.txt
knigtok/Глава в которой Бондарчук обыгрывает Толстого на Бородинском поле а Кубрик снимает лучший костюмный фильм в истории.txt
knigtok/Глава в которой Властелин колец становится экоманифестом а Толкин оказывается Гретой Тунберг 1950х.txt
knigtok/Глава в которой Галина Юзефович и Антон Долин возвращаются с сезоном об экранизациях — теперь с видеоверсией И начинают с Шерлока .txt
knigtok/Глава в которой Галина Юзефович советует читать на каникулах детективы а Антон Долин отговаривает смотреть Реальную любовь.txt
knigtok/Глава в которой Гамлет Фотоувеличение и даже Царь Эдип больше похожи на детектив чем сериал Шерлок.txt
knigtok/Глава в которой Гарри Поттер становится Христом и навсегда входит в наш культурный код хотите вы этого или нет.txt
knigtok/Глава в кот

In [13]:
from IPython.display import FileLink
FileLink(r'knizhnyy-bazar.tar.gz')

/kaggle/working/knigtok.tar.gz